<a href="https://colab.research.google.com/github/whbpt/3D_shape_alignment-rotation/blob/master/shape_alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the implementation to Construct  transform matrix to align any same shapes(such as triangle, tetrahedron..) together.
Here is the [reference](https://stackoverflow.com/questions/28075743/how-do-i-compose-a-rotation-matrix-with-human-readable-angles-from-scratch/28084380#28084380).

##library

In [0]:
########################
# loading library
########################
import numpy as np
import matplotlib.pylab as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [0]:
########################
# for 3D plotting
########################
init_notebook_mode(connected=False)
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
  <script src="/static/components/requirejs/require.js"></script>
  <script>requirejs.config({paths: {base: '/static/base', plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',},});</script>'''))
  
def plot_3D(X):
  configure_plotly_browser_state()
  cs = [[0.0,'blue'],[0.2,'cyan'],[0.4,'lime'],[0.6,'yellow'],[0.8,'orange'],[1.0,'red']]
  line_params = dict(width=20,color=np.arange(len(X)),colorscale=cs)
  iplot([go.Scatter3d(mode="lines",x=X[:,0],y=X[:,1],z=X[:,2],line=line_params)])

##main

In [0]:
#generate a random shape, n >=3
size_of_shape = 10 #@param {type:"number"}
ref_shape = np.random.random(size=(size_of_shape,3))

In [0]:
#generate a random transform shape from ref_shape

#rotation matrix
ang = np.random.random()

R = [[1,0,0],
     [0,np.cos(ang),-np.sin(ang)],
     [0,np.sin(ang),np.cos(ang)]]
R = np.array(R)

#translation matrix
T = np.random.random(size=(1,3))
new_shape = ref_shape@R + T

In [0]:
plot_3D(np.concatenate((new_shape)))

In [0]:
def transform(a,c):

  #normalization function by last axis
  def L2normize(x):
    return x/np.sqrt(np.sum(np.square(x),axis=-1,keepdims=True))
  
  #get transform matrix
  def get_m(c):
    x = c[1]
    y = c[2]
    z = np.cross(x,y)
    y = np.cross(z,x)
    return L2normize(np.stack((x,y,z),axis=0))
 

  return np.linalg.inv(get_m(a)) @ get_m(c)



![transform matrix](https://i.stack.imgur.com/1JvL0.png)

##show

In [0]:

R1 = transform(ref_shape - ref_shape[0],new_shape - new_shape[0])

a2c = (ref_shape - ref_shape[0])@R1 + new_shape[0]

plot_3D(np.concatenate((a2c,new_shape)))

In [0]:
R2 = transform(new_shape - new_shape[0],ref_shape - ref_shape[0])

c2a = (new_shape - new_shape[0])@R2 + ref_shape[0]

plot_3D(np.concatenate((c2a,ref_shape)))

## funky rotation

In [0]:
from matplotlib import animation,rc
rc('animation', html='html5')
#initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return (line,)
# animation function. This is called sequentially
ang = np.random.random()
def rotate(ang):
  R = [[1,0,0],
       [0,np.cos(ang),-np.sin(ang)],
       [0,np.sin(ang),np.cos(ang)]]
  return np.array(R)
def animate(i):
  
  new = ref_shape@rotate(i/np.pi*1)
  
  x = new[:,0]
  y = new[:,1]
  line.set_data(x, y)
  return (line,)
# call the animator. blit=True means only re-draw the parts that 
# have changed.
fig, ax = plt.subplots()
ax.set_xlim(( -2, 2))
ax.set_ylim((-2, 2))
line, = ax.plot([], [], lw=2)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=180, interval=100, blit=True)

In [0]:
anim